In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import functions
%matplotlib inline


# General information:
The midbootcamp project: Can we predict cancer using gene expression profile?

# To approach this objective:
1. select gene expression datasets for different types of cancers
2. Data checking, cleaning, transform if needed
3. Identify genes that are differentially expressed (called DEGs) in cancer samples vs normal samples. Using two samples t-test at p_sig = 0.05
4. Check and excluding genes that are highly correlated among the identified DEGs subset with a threshold for exluding at 0.95
5. Split and train model on working datasets:
    * using transformed data (using quantile transformation) vs non-transformed data 
6. Validation:
   * on the whole (train + test) dataset
   * using a new dataset of the same cancer type
   * calculation all validation metrics: precision, accuracy, recel, F1, cohen_kappa_score
     
# From the GEO database, we selected 3 expression profiling datasets for 3 types of cancers: prostate, breats, leukemia
1. link for download gene expression dataset: https://sbcb.inf.ufrgs.br/cumida
2. every dataset has it unique identifier as accession number GSE
3. For breast cancer dataset, accession number is GSE22820
    Description: it is expression profiling by array, generated from 176 primary breast cancer patients and 10 normal breast samples.
NOTE: however, when loading this dataset, there are only 129 primary breast cancer patients and 10 normal breast samples.
This should not affect the analysis.
4. since these datasets are repositories from different research groups to NCBI database (GEO), the format and setup is already standardized. Therefore, most likely, it will not require a cleaning process.
     

In [ ]:
# load the gene expression dataset GSE22820

path = '/Users/minhnguyen/IronHack2023-2024/Bootcamp/Labs/5_Mid_bootcamp_project_venv/data/Breast_GSE22820.csv'
df = pd.read_csv(path)
df.head(10)

In [ ]:
df.isnull().sum()

In [ ]:
# change data format using melt, so that can be used in boxplot

melted_data = df.melt(id_vars=['samples', 'type'], var_name='gene', value_name='expression')
melted_data
    

In [ ]:

# Create a boxplot with melted data

plt.figure(figsize=(20, 8))  
sns.boxplot(x='samples', y='expression', hue = 'type',  data=melted_data, width = 0.3)
plt.xticks(rotation=90)  

# Show the plot
plt.tight_layout()
plt.show()


# Testing 1: apply log transformation on original expression value

In [ ]:
# apply log transformation to data
melted_data_log = melted_data.copy()
melted_data_log['expression'] = melted_data_log['expression'].apply(lambda x: np.log10(x))
melted_data_log.head()

In [ ]:
# plot log transformed data on boxplot

plt.figure(figsize=(20, 8))  # Adjust figure size if needed
sns.boxplot(x='samples', y='expression', hue = 'type',  data=melted_data_log, width = 0.3)
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability

# Show the plot
plt.tight_layout()
plt.show()

# Testing 2: apply QuantileTransformation, to stabilize variance within each sample's gene profile

In [ ]:
# make a copy of df
df_transformed = df.copy()

# apply quantile transformation to data

from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer (output_distribution ='normal')
qt.fit(df[df.columns[2:]])
df_transformed[df.columns[2:]] = qt.transform(df[df.columns[2:]])
#melted_data_log['expression'] = melted_data_log['expression'].apply(lambda x: np.log10(x))
#melted_data_log.head()

In [ ]:
melted_df_transformed = df_transformed.melt(id_vars=['samples', 'type'], var_name='gene', value_name='expression')
melted_df_transformed

In [ ]:
# plot log transformed data on boxplot

plt.figure(figsize=(20, 8))  # Adjust figure size if needed
sns.boxplot(x='samples', y='expression', hue = 'type',  data=melted_df_transformed, width = 0.3)
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
df.iloc[0,3:].max()

normal = df[df['type'] == 'normal']
df.groupby(df['type']).mean()

In [ ]:
import scipy.stats as st

In [ ]:
df_cancer = df[df['type'] == "primary_breast_cancer"]
df_normal = df[df['type'] == "normal"]
df_cancer.head()
display(df_cancer.shape)
df_normal.head()
display(df_normal.shape)

In [ ]:
df_normal_melted = df_normal.melt(id_vars=['samples', 'type'], var_name='gene', value_name='expression')
sns.boxplot(df_normal_melted, x='samples', y='expression')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(5,2, figsize = (10,5 * 5))
row_index = 0
col_index = 0
for i in range(10):
    sns.histplot(df_normal.iloc[i,3:], ax = ax[row_index, col_index])
    ax[row_index, col_index].set_xlabel(df_normal.iloc[i,0])
    col_index +=1
    if col_index >1:
        col_index = 0
        row_index +=1
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(5,2, figsize = (10,5 * 5))
row_index = 0
col_index = 0
for i in range(10):
    sns.histplot(df_cancer.iloc[i,3:], ax = ax[row_index, col_index])
    ax[row_index, col_index].set_xlabel(df_normal.iloc[i,0])
    col_index +=1
    if col_index >1:
        col_index = 0
        row_index +=1
plt.tight_layout()
plt.show()
    

In [ ]:
df_cancer.columns[2:]

In [ ]:
p_sig = 0.05
for col in df_cancer.columns[2:5]:
    t, pvalue= st.ttest_ind(df_cancer[col],df_normal[col], equal_var = False, alternative = 'two-sided')
    print(t, pvalue)
    if pvalue < p_sig:
        print("df_cancer_mean is not equal df_normal_mean")
    else:
        print("there is no difference between 2 populations")

In [ ]:
p_sig = 0.05
differential_expressed_genes = []
for col in df_cancer.columns[2:]:
    t, pvalue= st.ttest_ind(df_cancer[col],df_normal[col], equal_var = False, alternative = 'two-sided')
    if pvalue < p_sig:
        differential_expressed_genes.append(str(col))
differential_expressed_genes 


In [ ]:
len(differential_expressed_genes)

In [ ]:
df['type']

In [ ]:
deg_df = df[['samples', 'type'] + differential_expressed_genes]
deg_df

In [ ]:
df[differential_expressed_genes]

In [ ]:
corr_matrix_df = df[differential_expressed_genes].corr()

In [ ]:
corr_matrix_df

In [ ]:
pd.DataFrame(corr_matrix_df.iterrows())
corr_matrix_df.row.items()

In [ ]:
def find_correlated_genes_optimized(r_values, threshold=0.95):
    correlated_genes_list = []

    # Create a mask for values above the threshold
    mask = (r_values.to_numpy() > threshold) & (r_values.index.to_numpy() != r_values.columns.to_numpy()[:, None])

    # Extract the column and index names where the mask is True
    correlated_columns, correlated_rows = np.where(mask)

    for col, index in zip(r_values.columns[correlated_columns], r_values.index[correlated_rows]):
        value = r_values.at[index, col]
        correlated_genes_list.append([col, value, index])

    return correlated_genes_list

In [ ]:
list_correlated_genes = []

for index, row in corr_matrix_df.iterrows():
    for col, value in row.items():

        if (value > 0.95) & (col != index):
            list_correlated_genes.append([col, value, index])

In [ ]:
display(list_correlated_genes)
len(list_correlated_genes)

In [ ]:
for col in df_cancer.columns[2:5]:
    print( (df_cancer[col].mean()) - (df_normal[col].mean())
    

In [ ]:
for col in df_cancer.columns[2:20]:
    t, pvalue= st.normaltest(df_cancer[col],df_normal[col].mean(), alternative = 'two-sided')
    print(t, pvalue)

In [ ]:
df_normal['NM_004900'].mean()

In [ ]:
df['type'].value_counts()


Qs
Monday morning (Explore) / afternoon Teusday Start answering
* What is the difference in GE profile between Tumoral and Normal
    * Are there any expression of interest (different between normal and tumoral?)
        - Does the mean/mode of the Tumoral and Normal compare, what are the differences?
        - Possible Bar graph from mean/mode from tumoral and Normal?
    * What is the overall differences between expression within the groups Tumoral/Normal?
        - Check if there are any outliers (possible high values per sample)
        - Check if something is overexpressed (higher than mean of normal) is this overall in tumoral, or only spiked in one sample
        - Possible Normalized line graph (sample to samplem, tumoral expression - control normal expression)
        - come up with maybe another to do visualize this?
    * Are there any genes correlated with eachtother (possible same pathway)
        - Check corraltion matrix on numericals
        - Pick 5 ten of correlated Genes/Probes
        - (Optionally) We can check some papers if really correlated
>> Might possible need some cleaning here?
    Are the genes between data sets the same?
Wednesday / Thursday
* What are the differences between to different types of Tumoral datasets?
    * Are the interesting picks the same between the different sets?
Thursday Morning
Prep the slides
- Dogma
- Why, Data science >3 min
- Grpahs >3 min

In [ ]:
meta = pd.read_csv('/Users/minhnguyen/Downloads/GPL6480_limpo.txt', delimiter = '\t')
meta.head()

In [ ]:
meta['GO_ID'].value_counts()